In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from model.config import cfg
from model.ass_fun import *
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim import losses
from tensorflow.contrib.slim import arg_scope
from tensorflow.contrib.slim import losses
from tensorflow.contrib.slim import arg_scope
from tensorflow.contrib.slim.python.slim.nets import resnet_utils
from tensorflow.contrib.slim.python.slim.nets import resnet_v1
from tensorflow.contrib.slim.python.slim.nets.resnet_v1 import resnet_v1_block
import os
import os.path as osp
import numpy as np
# `pip install easydict` if you don't have it
from easydict import EasyDict as edict
#______________________________________________

__C = edict()

cfg = __C

__C.DIR = 'D:/Semnan Uni/Visual Relationships/New folder/vrd-master'

__C.DIR2 = '/home/jhmei/yangxu/'

__C.POOLING_SIZE = 7

#VG
############################
__C.VG_BATCH_NUM = 128
__C.VG_BATCH_NUM2 = 30
__C.VG_NUM_CLASS = 201
__C.VG_NUM_RELA = 100
__C.VG_LR_INIT = 0.00001
__C.VG_TRAIN_ROUND = 30
__C.VG_RBM_SPARSITY_LAMBDA = 0.001

__C.VG_BATCH_NUM_RELA = 128
__C.VG_AU_PAIR = 5
__C.VG_IOU_TRAIN = 0.5
__C.VG_IOU_TEST = 0.5
############################

#VRD
############################
__C.VRD_BATCH_NUM = 30
__C.VRD_NUM_CLASS = 101
__C.VRD_NUM_RELA = 70
__C.VRD_LR_INIT = 0.00001
__C.VRD_TRAIN_ROUND = 20

__C.VRD_BATCH_NUM_RELA = 50
__C.VRD_AU_PAIR = 5
__C.VRD_IOU_TRAIN = 0.5
__C.VRD_IOU_TEST = 0.5
###########################

__C.IM_SIZE = 600
__C.IM_MAX_SIZE = 1000

__C.TRAIN = edict()

__C.TRAIN.WEIGHT_DECAY = 0.0005


__C.RESNET = edict()

__C.RESNET.FIXED_BLOCKS = 1

__C.VTR = edict()

#the dimension of embedding space in vtranse
__C.VTR.VG_R = 500

import numpy as np
import cv2
import os
from model.config import cfg


def generate_batch(N_total, N_each):
    """
    This file is used to generate index of the training batch.

    Arg:
        N_total:
        N_each:
    out_put:
        index_box: the corresponding index
    """
    num_batch = np.int32(N_total / N_each)
    if N_total % N_each == 0:
        index_box = range(N_total)
    else:
        index_box = np.empty(shape=[N_each * (num_batch + 1)], dtype=np.int32)
        index_box[0:N_total] = range(N_total)
        N_rest = N_each * (num_batch + 1) - N_total
        index_box[N_total:] = np.random.randint(0, N_total, N_rest)
    return index_box


def generate_batch_bal(labels, N_each):
    N_total = len(labels)
    num_batch = np.int32(N_total / N_each)
    if N_total % N_each == 0:
        index_box = range(N_total)
    else:
        index_box = np.empty(shape=[N_each * (num_batch + 1)], dtype=np.int32)
        index_box[0:N_total] = range(N_total)
        N_rest = N_each * (num_batch + 1) - N_total

        unique_labels = np.unique(labels, axis=0)
        N_unique = len(unique_labels)
        num_label = np.zeros([N_unique, ])
        for ii in range(N_unique):
            num_label[ii] = np.sum(labels == unique_labels[ii])
        prob_label = np.sum(num_label) / num_label
        prob_label = prob_label / np.sum(prob_label)
        index_rest = np.random.choice(N_unique, size=[N_rest, ], p=prob_label)
        for ii in range(N_rest):
            ind = index_rest[ii]
            ind2 = np.where(labels == unique_labels[ind])[0]
            a = np.random.randint(len(ind2))
            index_box[N_total + ii] = ind2[a]
    return index_box


def read_roidb(roidb_path):
    roidb_file = np.load(roidb_path)
    key = roidb_file.keys()[0]
    roidb_temp = roidb_file[key]
    roidb = roidb_temp[()]
    return roidb


def compute_iou(box, proposal):
    """
    compute the IoU between box with proposal
    Arg:
        box: [x1,y1,x2,y2]
        proposal: N*4 matrix, each line is [p_x1,p_y1,p_x2,p_y2]
    output:
        IoU: N*1 matrix, every IoU[i] means the IoU between
             box with proposal[i,:]
    """
    len_proposal = np.shape(proposal)[0]
    IoU = np.empty([len_proposal, 1])
    for i in range(len_proposal):
        xA = max(box[0], proposal[i, 0])
        yA = max(box[1], proposal[i, 1])
        xB = min(box[2], proposal[i, 2])
        yB = min(box[3], proposal[i, 3])

        if xB < xA or yB < yA:
            IoU[i, 0] = 0
        else:
            area_I = (xB - xA) * (yB - yA)
            area1 = (box[2] - box[0]) * (box[3] - box[1])
            area2 = (proposal[i, 2] - proposal[i, 0]) * (proposal[i, 3] - proposal[i, 1])
            IoU[i, 0] = area_I / float(area1 + area2 - area_I)
    return IoU


def compute_iou_each(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    if xB < xA or yB < yA:
        IoU = 0
    else:
        area_I = (xB - xA) * (yB - yA)
        area1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
        area2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
        IoU = area_I / float(area1 + area2 - area_I)
    return IoU


def im_preprocess(image_path):
    image = cv2.imread(image_path)
    im_orig = image.astype(np.float32, copy=True)
    im_orig -= np.array([[[102.9801, 115.9465, 122.7717]]])

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    target_size = cfg.IM_SIZE
    max_size = cfg.IM_MAX_SIZE
    im_scale = float(target_size) / float(im_size_min)

    if np.round(im_scale * im_size_max) > max_size:
        im_scale = float(max_size) / float(im_size_max)
    im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,
                    interpolation=cv2.INTER_LINEAR)
    im_shape_new = np.shape(im)
    im_use = np.zeros([1, im_shape_new[0], im_shape_new[1], im_shape_new[2]])
    im_use[0, :, :, :] = im
    return im_use, im_scale


def get_blob_pred(roidb_use, im_scale, index_sp, N_each_batch, batch_id):
    blob = {}
    sub_box = roidb_use['sub_box_gt'] * im_scale
    obj_box = roidb_use['obj_box_gt'] * im_scale
    rela = np.int32(roidb_use['rela_gt'])
    index = roidb_use['index_pred']

    index_use = index[batch_id * N_each_batch: (batch_id + 1) * N_each_batch]
    sub_box_use = sub_box[index_use, :]
    obj_box_use = obj_box[index_use, :]
    rela_use = rela[index_use]

    blob['sub_box'] = sub_box_use
    blob['obj_box'] = obj_box_use
    blob['rela'] = rela_use
    return blob


def get_blob_rela(roidb_use, im_scale, index_sp, N_each_batch, batch_id):
    blob = {}
    sub_box = roidb_use['sub_box_dete'] * im_scale
    obj_box = roidb_use['obj_box_dete'] * im_scale
    rela = np.int32(roidb_use['rela_dete'])
    index = roidb_use['index_rela']

    index_use = index[batch_id * N_each_batch: (batch_id + 1) * N_each_batch]
    sub_box_use = sub_box[index_use, :]
    obj_box_use = obj_box[index_use, :]
    rela_use = rela[index_use]

    blob['sub_box'] = sub_box_use
    blob['obj_box'] = obj_box_use
    blob['rela'] = rela_use
    return blob


def pred_recall(test_roidb, pred_roidb, N_recall):
    N_right = 0.0
    N_total = 0.0
    N_data = len(test_roidb)
    for i in range(N_data):
        gt_rela = test_roidb[i]['rela_gt']
        if len(gt_rela) == 0:
            continue
        pred_rela = pred_roidb[i]['pred_rela']
        pred_rela_score = pred_roidb[i]['pred_rela_score']
        N_rela = len(gt_rela)
        N_total = N_total + N_rela
        if N_rela <= N_recall:
            N_right = N_right + np.sum(np.float32(gt_rela == pred_rela))
        else:
            sort_score = np.sort(pred_rela_score)[::-1]
            thresh = sort_score[N_recall]
            for j in range(N_rela):
                if pred_rela_score[j] >= thresh:
                    N_right = N_right + np.float32(gt_rela[j] == pred_rela[j])

    acc = N_right / N_total
    return acc


def rela_recall(test_roidb, pred_roidb, N_recall):
    N_right = 0.0
    N_total = 0.0
    N_data = len(test_roidb)
    num_right = np.zeros([N_data, ])
    for i in range(N_data):
        rela_gt = test_roidb[i]['rela_gt']
        if len(rela_gt) == 0:
            continue
        sub_gt = test_roidb[i]['sub_gt']
        obj_gt = test_roidb[i]['obj_gt']
        sub_box_gt = test_roidb[i]['sub_box_gt']
        obj_box_gt = test_roidb[i]['obj_box_gt']

        pred_rela = pred_roidb[i]['pred_rela']
        pred_rela_score = pred_roidb[i]['pred_rela_score']
        sub_dete = pred_roidb[i]['sub_dete']
        obj_dete = pred_roidb[i]['obj_dete']
        sub_box_dete = pred_roidb[i]['sub_box_dete']
        obj_box_dete = pred_roidb[i]['obj_box_dete']

        N_rela = len(rela_gt)
        N_total = N_total + N_rela

        N_pred = len(pred_rela)

        sort_score = np.sort(pred_rela_score)[::-1]
        if N_recall >= N_pred:
            thresh = -1
        else:
            thresh = sort_score[N_recall]

        detected_gt = np.zeros([N_rela, ])
        for j in range(N_pred):
            if pred_rela_score[j] <= thresh:
                continue

            for k in range(N_rela):
                if detected_gt[k] == 1:
                    continue
                if (sub_gt[k] == sub_dete[j]) and (obj_gt[k] == obj_dete[j]) and (rela_gt[k] == pred_rela[j]):
                    s_iou = compute_iou_each(sub_box_dete[j], sub_box_gt[k])
                    o_iou = compute_iou_each(obj_box_dete[j], obj_box_gt[k])
                    if (s_iou >= 0.5) and (o_iou >= 0.5):
                        detected_gt[k] = 1
                        N_right = N_right + 1
                        num_right[i] = num_right[i] + 1

    acc = N_right / N_total
    print(N_right)
    print(N_total)
    return acc, num_right


def phrase_recall(test_roidb, pred_roidb, N_recall):
    N_right = 0.0
    N_total = 0.0
    N_data = len(test_roidb)
    num_right = np.zeros([N_data, ])
    for i in range(N_data):
        rela_gt = test_roidb[i]['rela_gt']
        if len(rela_gt) == 0:
            continue

        N_rela = len(rela_gt)
        sub_gt = test_roidb[i]['sub_gt']
        obj_gt = test_roidb[i]['obj_gt']
        sub_box_gt = test_roidb[i]['sub_box_gt']
        obj_box_gt = test_roidb[i]['obj_box_gt']
        phrase_gt = generate_phrase_box(sub_box_gt, obj_box_gt)

        pred_rela = pred_roidb[i]['pred_rela']
        pred_rela_score = pred_roidb[i]['pred_rela_score']
        sub_dete = pred_roidb[i]['sub_dete']
        obj_dete = pred_roidb[i]['obj_dete']
        sub_box_dete = pred_roidb[i]['sub_box_dete']
        obj_box_dete = pred_roidb[i]['obj_box_dete']
        phrase_dete = generate_phrase_box(sub_box_dete, obj_box_dete)
        N_pred = len(pred_rela)

        N_total = N_total + N_rela

        sort_score = np.sort(pred_rela_score)[::-1]
        if N_recall >= N_pred:
            thresh = -1
        else:
            thresh = sort_score[N_recall]

        detected_gt = np.zeros([N_rela, ])
        for j in range(N_pred):
            if pred_rela_score[j] <= thresh:
                continue

            for k in range(N_rela):
                if detected_gt[k] == 1:
                    continue
                if (sub_gt[k] == sub_dete[j]) and (obj_gt[k] == obj_dete[j]) and (rela_gt[k] == pred_rela[j]):
                    iou = compute_iou_each(phrase_dete[j], phrase_gt[k])
                    if iou >= 0.5:
                        detected_gt[k] = 1
                        N_right = N_right + 1
                        num_right[i] = num_right[i] + 1

    acc = N_right / N_total
    print(N_right)
    print(N_total)
    return acc, num_right


def print_pred_res(test_roidb, pred_roidb, res_name, N_rela):
    N_pred_rela = np.zeros([N_rela, ])
    N_right_rela = np.zeros([N_rela, ])
    N_gt_rela = np.zeros([N_rela, ])
    N_acc = np.zeros([N_rela, ])
    N_pred_other = np.zeros([N_rela, N_rela])

    N_pred = len(pred_roidb)
    for i in range(N_pred):
        pred_rela = np.int32(pred_roidb[i]['pred_rela'])
        gt_rela = np.int32(test_roidb[i]['rela_gt'])
        for k in range(len(gt_rela)):
            N_pred_rela[pred_rela[k]] += 1
            N_gt_rela[gt_rela[k]] += 1
            N_pred_other[gt_rela[k]][pred_rela[k]] += 1
            if pred_rela[k] == gt_rela[k]:
                N_right_rela[gt_rela[k]] += 1

    for k in range(N_rela):
        if N_gt_rela[k] == 0:
            N_acc[k] = -1
            continue
        N_acc[k] = (N_right_rela[k] + 0.0) / N_gt_rela[k]

    text_file = open(res_name, "aw")
    for k in range(N_rela):
        text_file.write('k: {0}, acc: {1}\n'.format(k, N_acc[k]))
    for k in range(N_rela):
        text_file.write('k: {0}, num of pred: {1}\n'.format(k, N_pred_rela[k]))
    for k in range(N_rela):
        text_file.write('k: {0}, others: {1}\n'.format(k, np.where(N_pred_other[k] > 0)[0]))
    text_file.close()


def generate_phrase_box(sbox, obox):
    N_box = len(sbox)
    phrase = np.zeros([N_box, 4])
    for i in range(N_box):
        phrase[i, 0] = min(sbox[i, 0], obox[i, 0])
        phrase[i, 1] = min(sbox[i, 1], obox[i, 1])
        phrase[i, 2] = max(sbox[i, 2], obox[i, 2])
        phrase[i, 3] = max(sbox[i, 2], obox[i, 3])
    return phrase


def extract_detected_box(dete_box):
    N_cls = len(dete_box['pred_boxes'])
    pred_box = dete_box['pred_boxes']
    detected_cls = np.zeros([1000, ])
    detected_box = np.zeros([1000, 4])
    detected_score = np.zeros([1000, ])
    t = 0
    for i in range(N_cls):
        l = len(pred_box[i])
        if l == 0:
            continue
        box_temp = pred_box[i]
        detected_box[t:t + l] = box_temp[:, 0:4]
        detected_cls[t:t + l] = i
        detected_score[t:t + l] = box_temp[:, 4]
        t = t + l
    detected_box = detected_box[0:t]
    detected_cls = detected_cls[0:t]
    detected_score = detected_score[0:t]
    return detected_box, detected_cls, detected_score


def extract_roidb_box(roidb):
    sbox = roidb['sub_box_gt']
    obox = roidb['obj_box_gt']
    roidb_box = np.concatenate((roidb['sub_box_gt'], roidb['obj_box_gt']), axis=0)
    roidb_cls = np.concatenate((roidb['sub_gt'], roidb['obj_gt']), axis=0)
    roidb_rela = roidb['rela_gt']
    N_rela = len(roidb_rela)
    unique_boxes, unique_inds = np.unique(roidb_box, axis=0, return_index=True)
    unique_cls = roidb_cls[unique_inds]
    rela_box_index = np.zeros([N_rela, 2])
    for i in range(N_rela):
        rela_box_index[i, 0] = np.where(unique_boxes == sbox[i])[0][0]
        rela_box_index[i, 1] = np.where(unique_boxes == obox[i])[0][0]
    return unique_cls, unique_boxes, roidb_rela, rela_box_index


def generate_au_box(unique_boxes, detected_box, iou_l):
    N_unique = len(unique_boxes)
    au_box = []
    for i in range(N_unique):
        box_temp = unique_boxes[i]
        iou = compute_iou(box_temp, detected_box)
        index_temp = np.where(iou > iou_l)[0]
        box_use = detected_box[index_temp]
        box_use = np.vstack((box_use, box_temp))
        au_box.append(box_use)
    return au_box


def generate_rela_info(au_box, index, N_each_pair):
    s_id = np.int32(index[0])
    o_id = np.int32(index[1])
    sbox = au_box[s_id]
    obox = au_box[o_id]
    N_s = len(sbox)
    N_o = len(obox)
    sa = np.random.randint(0, N_s, [N_each_pair, ])
    oa = np.random.randint(0, N_o, [N_each_pair, ])
    sbox_use = sbox[sa]
    obox_use = obox[oa]
    return sbox_use, obox_use


def generate_train_rela_roidb(roidb, dete_box, iou_l, N_each_batch, N_each_pair):
    unique_cls, unique_boxes, roidb_rela, rela_box_index = extract_roidb_box(roidb)
    detected_box, detected_cls, detected_score = extract_detected_box(dete_box)
    au_box = generate_au_box(unique_boxes, detected_box, iou_l)

    N_rela = len(roidb_rela)
    for i in range(N_rela):
        sbox, obox = generate_rela_info(au_box, rela_box_index[i], N_each_pair)
        s_id = np.int32(rela_box_index[i, 0])
        o_id = np.int32(rela_box_index[i, 1])
        sb = np.zeros([N_each_pair, ]) + unique_cls[s_id]
        ob = np.zeros([N_each_pair, ]) + unique_cls[o_id]
        rela = np.zeros([N_each_pair, ]) + roidb_rela[i]
        if i == 0:
            sub_box_dete = sbox
            obj_box_dete = obox
            sub_dete = sb
            obj_dete = ob
            rela_dete = rela
        else:
            sub_box_dete = np.vstack((sub_box_dete, sbox))
            obj_box_dete = np.vstack((obj_box_dete, obox))
            sub_dete = np.concatenate((sub_dete, sb), axis=0)
            obj_dete = np.concatenate((obj_dete, ob), axis=0)
            rela_dete = np.concatenate((rela_dete, rela), axis=0)

    index_rela = generate_batch(len(rela_dete), N_each_batch)

    roidb_temp = {'image': roidb['image'], 'au_box': au_box, 'rela_box_index': rela_box_index,
                  'unique_cls': unique_cls, 'unique_box': unique_boxes, 'rela_gt': roidb_rela,
                  'sub_box_dete': sub_box_dete, 'obj_box_dete': obj_box_dete, 'sub_dete': sub_dete,
                  'obj_dete': obj_dete, 'rela_dete': rela_dete, 'index_rela': index_rela}
    return roidb_temp


def generate_test_rela_roidb(roidb, dete_box, N_each_batch):
    detected_box, detected_cls, detected_score = extract_detected_box(dete_box)
    N_dete = len(detected_box)
    sub_box_dete = np.zeros([N_dete * (N_dete - 1), 4])
    obj_box_dete = np.zeros([N_dete * (N_dete - 1), 4])
    sub_dete = np.zeros([N_dete * (N_dete - 1), ])
    obj_dete = np.zeros([N_dete * (N_dete - 1), ])
    sub_score = np.zeros([N_dete * (N_dete - 1), ])
    obj_score = np.zeros([N_dete * (N_dete - 1), ])

    t = 0
    for i in range(N_dete):
        for j in range(N_dete):
            if j == i:
                continue
            sub_box_dete[t] = detected_box[i]
            obj_box_dete[t] = detected_box[j]
            sub_dete[t] = detected_cls[i]
            obj_dete[t] = detected_cls[j]
            sub_score[t] = detected_score[i]
            obj_score[t] = detected_score[j]
            t = t + 1
    sub_box_dete = sub_box_dete[0:t]
    obj_box_dete = obj_box_dete[0:t]
    sub_dete = sub_dete[0:t]
    obj_dete = obj_dete[0:t]
    sub_score = sub_score[0:t]
    obj_score = obj_score[0:t]
    index_rela = generate_batch(len(sub_box_dete), N_each_batch)
    rela_dete = np.zeros([len(sub_box_dete), ])

    roidb_temp = {'image': roidb['image'], 'rela_gt': roidb['rela_gt'], 'sub_gt': roidb['sub_gt'],
                  'obj_gt': roidb['obj_gt'],
                  'sub_box_dete': sub_box_dete, 'obj_box_dete': obj_box_dete, 'sub_dete': sub_dete,
                  'obj_dete': obj_dete,
                  'detected_box': detected_box, 'detected_cls': detected_cls, 'sub_box_gt': roidb['sub_box_gt'],
                  'obj_box_gt': roidb['obj_box_gt'], 'sub_score': sub_score, 'obj_score': obj_score,
                  'index_rela': index_rela,
                  'rela_dete': rela_dete}

    return roidb_temp


class VTranse(object):
	def __init__(self):
		self.predictions = {}
		self.losses = {}
		self.layers = {}
		self.feat_stride = [16, ]
		self.scope = 'vgg_16'

	def create_graph(self, N_each_batch, index_sp, index_cls, num_classes, num_predicates):
		self.image = tf.placeholder(tf.float32, shape=[1, None, None, 3])
		self.sbox = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
		self.obox = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
		self.sub_sp_info = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
		self.ob_sp_info = tf.placeholder(tf.float32, shape=[N_each_batch, 4])
		self.rela_label = tf.placeholder(tf.int32, shape=[N_each_batch,])
		self.keep_prob = tf.placeholder(tf.float32)
		self.index_sp = index_sp
		self.index_cls = index_cls
		self.num_classes = num_classes
		self.num_predicates = num_predicates
		self.N_each_batch = N_each_batch

		self.build_dete_network()
		self.build_rd_network()
		self.add_rd_loss()


	def build_dete_network(self, is_training=True):
		net_conv = self.image_to_head(is_training)
		sub_pool5 = self.crop_pool_layer(net_conv, self.sbox, "sub_pool5")
		ob_pool5 = self.crop_pool_layer(net_conv, self.obox, "ob_pool5")
		sub_fc7 = self.head_to_tail(sub_pool5, is_training, reuse = False)
		ob_fc7 = self.head_to_tail(ob_pool5, is_training, reuse = True)

		with tf.variable_scope(self.scope, self.scope):
			# region classification
			sub_cls_prob, sub_cls_pred = self.region_classification(sub_fc7, is_training, reuse = False)
		with tf.variable_scope(self.scope, self.scope):
			# region classification
			ob_cls_prob, ob_cls_pred = self.region_classification(ob_fc7, is_training, reuse = True)

		self.predictions['sub_cls_prob'] = sub_cls_prob
		self.predictions['sub_cls_pred'] = sub_cls_pred
		self.predictions['ob_cls_prob'] = ob_cls_prob
		self.predictions['ob_cls_pred'] = ob_cls_pred
		self.layers['sub_pool5'] = sub_pool5
		self.layers['ob_pool5'] = ob_pool5
		self.layers['sub_fc7'] = sub_fc7
		self.layers['ob_fc7'] = ob_fc7

	def image_to_head(self, is_training, reuse=False):
		with tf.variable_scope(self.scope, self.scope, reuse=reuse):
			net = slim.repeat(self.image, 2, slim.conv2d, 64, [3, 3],
				trainable=is_training, scope='conv1')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool1')
			net = slim.repeat(net, 2, slim.conv2d, 128, [3, 3],
				trainable=is_training, scope='conv2')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool2')
			net = slim.repeat(net, 3, slim.conv2d, 256, [3, 3],
				trainable=is_training, scope='conv3')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool3')
			net = slim.repeat(net, 3, slim.conv2d, 512, [3, 3],
				trainable=is_training, scope='conv4')
			net = slim.max_pool2d(net, [2, 2], padding='SAME', scope='pool4')
			net_conv = slim.repeat(net, 3, slim.conv2d, 512, [3, 3],
				trainable=is_training, scope='conv5')

			self.layers['head'] = net_conv
			return net_conv

	def head_to_tail(self, pool5, is_training, reuse=False):
		with tf.variable_scope(self.scope, self.scope, reuse=reuse):
			pool5_flat = slim.flatten(pool5, scope='flatten')
			fc6 = slim.fully_connected(pool5_flat, 4096, scope='fc6')
			fc6 = slim.dropout(fc6, keep_prob=self.keep_prob, is_training=True,
					scope='dropout6')
			fc7 = slim.fully_connected(fc6, 4096, scope='fc7')
			fc7 = slim.dropout(fc7, keep_prob=self.keep_prob, is_training=True,
					scope='dropout7')

			return fc7

	def crop_pool_layer(self, bottom, rois, name):
		"""
		Notice that the input rois is a N*4 matrix, and the coordinates of x,y should be original x,y times im_scale.
		"""
		with tf.variable_scope(name) as scope:
			n=tf.to_int32(rois.shape[0])
			batch_ids = tf.zeros([n,],dtype=tf.int32)
			# Get the normalized coordinates of bboxes
			bottom_shape = tf.shape(bottom)
			height = (tf.to_float(bottom_shape[1]) - 1.) * np.float32(self.feat_stride[0])
			width = (tf.to_float(bottom_shape[2]) - 1.) * np.float32(self.feat_stride[0])
			x1 = tf.slice(rois, [0, 0], [-1, 1], name="x1") / width
			y1 = tf.slice(rois, [0, 1], [-1, 1], name="y1") / height
			x2 = tf.slice(rois, [0, 2], [-1, 1], name="x2") / width
			y2 = tf.slice(rois, [0, 3], [-1, 1], name="y2") / height
			# Won't be back-propagated to rois anyway, but to save time
			bboxes = tf.stop_gradient(tf.concat([y1, x1, y2, x2], 1))
			crops = tf.image.crop_and_resize(bottom, bboxes, tf.to_int32(batch_ids), [cfg.POOLING_SIZE*2, cfg.POOLING_SIZE*2], method='bilinear',
											 name="crops")
			pooling = max_pool(crops, 2, 2, 2, 2, name="max_pooling")
		return pooling


	def region_classification(self, fc7, is_training, reuse = False):
		cls_score = slim.fully_connected(fc7, self.num_classes,
										 activation_fn=None, scope='cls_score', reuse=reuse)
		print("cls_score's shape: {0}".format(cls_score.get_shape()))
		cls_prob = tf.nn.softmax(cls_score, name="cls_prob")
		cls_pred = tf.argmax(cls_score, axis=1, name="cls_pred")

		return cls_prob, cls_pred

	def build_rd_network(self):
		sub_sp_info = self.sub_sp_info
		ob_sp_info = self.ob_sp_info
		sub_cls_prob = self.predictions['sub_cls_prob']
		ob_cls_prob = self.predictions['ob_cls_prob']
		sub_fc = self.layers['sub_fc7']
		ob_fc = self.layers['ob_fc7']

		if self.index_sp:
			sub_fc = tf.concat([sub_fc, sub_sp_info], axis = 1)
			ob_fc = tf.concat([ob_fc, ob_sp_info], axis = 1)
		if self.index_cls:
			sub_fc = tf.concat([sub_fc, sub_cls_prob], axis = 1)
			ob_fc = tf.concat([ob_fc, ob_cls_prob], axis = 1)

		sub_fc1 = slim.fully_connected(sub_fc, cfg.VTR.VG_R,
										 activation_fn=tf.nn.relu, scope='RD_sub_fc1')
		ob_fc1 = slim.fully_connected(ob_fc, cfg.VTR.VG_R,
										 activation_fn=tf.nn.relu, scope='RD_ob_fc1')
		dif_fc1 = ob_fc1 - sub_fc1
		rela_score = slim.fully_connected(dif_fc1, self.num_predicates,
										 activation_fn=None, scope='RD_fc2')
		rela_prob = tf.nn.softmax(rela_score)
		self.layers['rela_score'] = rela_score
		self.layers['rela_prob'] = rela_prob

	def add_rd_loss(self):
		rela_score = self.layers['rela_score']
		rela_prob = self.layers['rela_prob']
		rela_label = self.rela_label
		rd_loss = tf.reduce_mean( tf.nn.sparse_softmax_cross_entropy_with_logits(
									labels = rela_label, logits = rela_score) )
		self.losses['rd_loss'] = rd_loss

		acc_each = tf.nn.in_top_k(rela_score, rela_label, 1)
		self.losses['acc_each'] = acc_each
		self.losses['acc'] = tf.reduce_mean( tf.cast(acc_each, tf.float32) )

		rela_pred = tf.argmax(rela_score, 1)
		self.predictions['rela_pred'] = rela_pred

		rela_max_prob = tf.reduce_max(rela_prob, 1)
		self.predictions['rela_max_prob'] = rela_max_prob

	def train_predicate(self, sess, roidb_use, RD_train):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_pred'])/self.N_each_batch
		RD_loss = 0.0
		acc = 0.0
		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 0.5}
			_, losses = sess.run([RD_train, self.losses], feed_dict = feed_dict)
			RD_loss = RD_loss + losses['rd_loss']
			acc = acc + losses['acc']

		RD_loss = RD_loss/batch_num
		acc = acc/batch_num
		return RD_loss, acc

	def train_rela(self, sess, roidb_use, RD_train):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_rela'])/self.N_each_batch
		RD_loss = 0.0
		acc = 0.0
		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_rela(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 0.5}
			_, losses = sess.run([RD_train, self.losses], feed_dict = feed_dict)
			RD_loss = RD_loss + losses['rd_loss']
			acc = acc + losses['acc']

		RD_loss = RD_loss/batch_num
		acc = acc/batch_num
		return RD_loss, acc

	def val_predicate(self, sess, roidb_use):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_pred'])/self.N_each_batch
		RD_loss = 0.0
		acc = 0.0
		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 1}
			losses = sess.run(self.losses, feed_dict = feed_dict)
			RD_loss = RD_loss + losses['rd_loss']
			acc = acc + losses['acc']

		RD_loss = RD_loss/batch_num
		acc = acc/batch_num
		return RD_loss, acc

	def val_rela(self, sess, roidb_use):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_rela'])/self.N_each_batch
		RD_loss = 0.0
		acc = 0.0
		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_rela(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 1}
			losses = sess.run(self.losses, feed_dict = feed_dict)
			RD_loss = RD_loss + losses['rd_loss']
			acc = acc + losses['acc']

		RD_loss = RD_loss/batch_num
		acc = acc/batch_num
		return RD_loss, acc

	def test_predicate(self, sess, roidb_use):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_pred'])/self.N_each_batch
		pred_rela = np.zeros([len(roidb_use['index_pred']),])
		pred_rela_score = np.zeros([len(roidb_use['index_pred']),])

		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_pred(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 1}
			predictions = sess.run(self.predictions, feed_dict = feed_dict)
			pred_rela[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_pred'][:]
			pred_rela_score[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_max_prob'][:]
		N_rela = len(roidb_use['rela_gt'])
		pred_rela = pred_rela[0:N_rela]
		pred_rela_score = pred_rela_score[0:N_rela]
		return pred_rela, pred_rela_score

	def test_rela(self, sess, roidb_use):
		im, im_scale = im_preprocess(roidb_use['image'])
		batch_num = len(roidb_use['index_rela'])/self.N_each_batch
		pred_rela = np.zeros([len(roidb_use['index_rela']),])
		pred_rela_score = np.zeros([len(roidb_use['index_rela']),])

		for batch_id in range(np.int32(batch_num)):
			blob = get_blob_rela(roidb_use, im_scale, self.index_sp, self.N_each_batch, batch_id)
			feed_dict = {self.image: im, self.sbox: blob['sub_box'], self.obox: blob['obj_box'], self.rela_label: blob['rela'],
						 self.keep_prob: 1}
			predictions = sess.run(self.predictions, feed_dict = feed_dict)
			pred_rela[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_pred'][:]
			pred_rela_score[batch_id*self.N_each_batch:(batch_id+1)*self.N_each_batch] = predictions['rela_max_prob'][:]
		N_rela = len(roidb_use['rela_dete'])
		pred_rela = pred_rela[0:N_rela]
		pred_rela_score = pred_rela_score[0:N_rela]
		return pred_rela, pred_rela_score

def conv(x, h, w, K, s_y, s_x, name, relu = True, reuse=False, padding='SAME'):
	"""
	Args:
		x: input
		h: height of filter
		w: width of filter
		K: number of filters
		s_y: stride of height of filter
		s_x: stride of width of filter
	"""
	#c means the number of input channels
	c = int(x.get_shape()[-1])

	with tf.variable_scope(name, reuse=reuse) as scope:
		weights = tf.get_variable('weights', shape=[h,w,c,K])
		biases = tf.get_variable('biases', shape=[K])

		conv_value = tf.nn.conv2d(x, weights, strides = [1,s_y,s_x,1], padding = padding)
		add_baises_value = tf.reshape(tf.nn.bias_add(conv_value, biases), tf.shape(conv_value))
		if relu==True:
			relu_value = tf.nn.relu(add_baises_value, name=scope.name)
		else:
			relu_value = add_baises_value

		return relu_value

def fc(x,K,name,relu=True,reuse=False):
	"""
	Args:
		x: input
		K: the dimension of the output
	"""
	#c means the number of input channels

	c = int(x.get_shape()[1])
	with tf.variable_scope(name, reuse=reuse) as scope:
		weights = tf.get_variable('weights', shape=[c,K])
		biases = tf.get_variable('biases',shape=[K])
		relu_value = tf.nn.xw_plus_b(x,weights,biases,name = scope.name)
		if relu:
			result_value = tf.nn.relu(relu_value)
		else:
			result_value = relu_value
		return result_value

def max_pool(x, h, w, s_y, s_x, name, padding='SAME'):
	return tf.nn.max_pool(x, ksize=[1,h,w,1], strides=[1, s_x, s_y, 1], padding=padding, name=name)

def avg_pool(x, h, w, s_y, s_x, name, padding='SAME'):
	return tf.nn.avg_pool(x, ksize=[1,h,w,1], strides=[1, s_x, s_y, 1], padding=padding, name=name)

def dropout(x, keep_prob):
	return tf.nn.dropout(x, keep_prob)

def leaky_relu(x, alpha):
	return tf.maximum(x, alpha * x)

#_________________________________________
N_cls = cfg.VRD_NUM_CLASS
N_rela = cfg.VRD_NUM_RELA
N_each_batch = cfg.VRD_BATCH_NUM
lr_init = cfg.VRD_LR_INIT


index_sp = False
index_cls = False

vnet = VTranse()
vnet.create_graph(N_each_batch, index_sp, index_cls, N_cls, N_rela)

roidb_path = cfg.DIR + 'input/vrd_roidb.npz'
res_path = cfg.DIR + 'pretrained_para/vrd_vgg_pretrained.ckpt'

roidb_read = read_roidb(roidb_path)
train_roidb = roidb_read['train_roidb']
test_roidb = roidb_read['test_roidb']
N_train = len(train_roidb)
N_test = len(test_roidb)
N_round = 10
N_show = 100
N_save = N_train
N_val = N_test

total_var = tf.trainable_variables()
restore_var = [var for var in total_var if 'vgg_16' in var.name]
cls_score_var = [var for var in total_var if 'cls_score' in var.name]
res_var = [item for item in restore_var if item not in cls_score_var]

saver_res = tf.train.Saver(var_list = restore_var)

RD_var = [var for var in total_var if 'RD' in var.name]

saver = tf.train.Saver(max_to_keep = 200)
for var in RD_var:
	print(var)

optimizer = tf.train.AdamOptimizer(learning_rate=lr_init)
train_loss = vnet.losses['rd_loss']
RD_train = optimizer.minimize(train_loss, var_list = RD_var)

with tf.Session() as sess:
	init = tf.global_variables_initializer()
	sess.run(init)
	saver_res.restore(sess, res_path)

	t = 0.0
	rd_loss = 0.0
	acc = 0.0
	for r in range(N_round):
		for roidb_id in range(N_train):
			roidb_use = train_roidb[roidb_id]
			if len(roidb_use['rela_gt']) == 0:
				continue
			rd_loss_temp, acc_temp = vnet.train_predicate(sess, roidb_use, RD_train)
			rd_loss = rd_loss + rd_loss_temp
			acc = acc + acc_temp
			t = t + 1.0
			if t % N_show == 0:
				print("t: {0}, rd_loss: {1}, acc: {2}".format(t, rd_loss/N_show, acc/N_show))
				rd_loss = 0.0
				acc = 0.0
			if t % N_save == 0:
				save_path = cfg.DIR + 'vtranse/pred_para/vrd_vgg/vrd_vgg' + format(int(t/N_save),'04') + '.ckpt'
				print("saving model to {0}".format(save_path))
				saver.save(sess, save_path)
				rd_loss_val = 0.0
				acc_val = 0.0
				for val_id in range(N_val):
					roidb_use = test_roidb[val_id]
					if len(roidb_use['rela_gt']) == 0:
						continue
					rd_loss_temp, acc_temp = vnet.val_predicate(sess, roidb_use)
					rd_loss_val = rd_loss_val + rd_loss_temp
					acc_val = acc_val + acc_temp
				print("val: rd_loss: {0}, acc: {1}".format(rd_loss_val/N_val, acc_val/N_val))